# Challenges:
* Encode companies
* Encode people
* Encode cities and countries
* use text data embedding (word/document) or word freq 
* Imputation

imports

In [174]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
from sklearn import preprocessing
from sklearn.impute import KNNImputer

Reviewing a sample row from each file

In [175]:
acquired = pd.read_csv("Data/Acquired Tech Companies.csv")
acquired.iloc[0]

Company                                                     Day Software
CrunchBase Profile     http://www.crunchbase.com/organization/day-sof...
Image                  http://a5.images.crunchbase.com/image/upload/c...
Tagline                Day Software develops web applications that al...
Year Founded                                                         NaN
Market Categories                                               Software
Address (HQ)           Barfüsserplatz 6, Basel, Basel-Stadt, Switzerland
City (HQ)                                                          Basel
State / Region (HQ)                                          Basel-Stadt
Country (HQ)                                                 Switzerland
Description            Day was founded in Basel, Switzerland, in 1993...
Homepage                                              http://www.day.com
Twitter                                                              NaN
Acquired by                                        

In [176]:
acquiring = pd.read_csv("Data/Acquiring Tech Companies.csv")
acquiring.iloc[0]

Acquiring Company                                                                        Adobe
CrunchBase Profile                               www.crunchbase.com/organization/adobe-systems
Image                                        http://a2.images.crunchbase.com/image/upload/c...
Tagline                                      Adobe is an American multinational computer so...
Market Categories                            Photo Editing, Design, Creative, Software, Ima...
Year Founded                                                                              1982
IPO                                                                                       1986
Founders                                                         John Warnock, Charles Geschke
Number of Employees                                                                     11,144
Number of Employees (year of last update)                                               2012.0
Total Funding ($)                                 

In [177]:
acquisitions = pd.read_csv("Data/Acquisitions.csv")
acquisitions.iloc[0]

Acquisitions ID                                      EMC acquired Data Domain in 2009
Acquired Company                                                          Data Domain
Acquiring Company                                                                 EMC
Year of acquisition announcement                                                 2009
Deal announced on                                                           8/07/2009
Price                                                                  $2,100,000,000
Status                                                                    Undisclosed
Terms                                                                            Cash
Acquisition Profile                 http://www.crunchbase.com/acquisition/5dc676a1...
News                                                         EMC acquired Data Domain
News Link                           http://www.businesswire.com/news/home/20090708...
Name: 0, dtype: object

In [178]:
founders = pd.read_csv("Data/Founders and Board Members.csv")
founders.iloc[0]

Name                                                 Hans-Werner Hector
CrunchBase Profile      http://de.wikipedia.org/wiki/Hans-Werner_Hector
Role                                                            Founder
Companies                                                           SAP
Image                 http://images.forbes.com/media/lists/10/2006/4...
Name: 0, dtype: object

We will link between the files using these columns:
* Acquisitions ID to link the acquisitions
* 'Founders' and 'Name' to link the Founders

In [179]:
np.intersect1d(acquired.columns, acquisitions.columns).tolist()

['Acquisitions ID']

In [180]:
np.intersect1d(acquiring.columns, acquisitions.columns).tolist()

['Acquiring Company', 'Acquisitions ID']

In [181]:
def ValidateLink(url, timeout=15):
    session = requests.Session()
    # fake headers to make it seem like a real request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "DNT": "1",
    }
    session.headers.update(headers)
    try:
        response = session.get(url, timeout=timeout, allow_redirects=True, stream=True)
        status_code = response.status_code
        response.close()
        if status_code < 400:
            return True
        else:
            return False
    except Exception as e:
        return False

In [182]:
def ValidateLinks(urls):
    results = []
    for url in urls:
        results.append(ValidateLink(url))
        if results[-1]:
            return results
    return results

In [183]:
def ValidateLinksDF(df):
    for col in df.columns:
        for val in df[col]:
            if type(val) == str and ("http" in val):
                print(col)
                results = ValidateLinks(df[col])
                if not pd.Series(results).any():
                    print(f'Column "{col}" had no valid links , or is using captcha.')
                    print("Try it yourself:")
                    print(df[col][0] + "\n")
                break

ValidateLinksDF(acquired)

* CrunchBase is using CAPTCHA , so we won't drop it now but we will process it later
* Image links are all corrupt so we will drop the column 

In [184]:
acquired = acquired.drop("Image", axis=1)

ValidateLinksDF(acquiring)

* drop Image also

In [185]:
acquiring = acquiring.drop("Image", axis=1)

ValidateLinksDF(acquisitions)

* acquisitions profile is also a crunchbase link

In [186]:
acquisitions = acquisitions.drop("Acquisition Profile", axis=1)

ValidateLinksDF(founders)

We don't need the exact address of the company, we already have the city , state and country

In [187]:
acquired = acquired.drop("Address (HQ)", axis=1)
acquiring = acquiring.drop("Address (HQ)", axis=1)

**Adding the target variable**

In [188]:
acquisitions["Price"] = [
    int(price.removeprefix("$").replace(",", "")) for price in acquisitions["Price"]
]

In [189]:
acquired["Price"] = None
acquired["Year of acquisition announcement"] = None

In [190]:
for i, company in enumerate(acquisitions["Acquired Company"]):
    acquired.loc[acquired["Company"] == company, "Price"] = acquisitions.iloc[i][
        "Price"
    ]
    acquired.loc[acquired["Company"] == company, "Year of acquisition announcement"] = (
        acquisitions.iloc[i]["Year of acquisition announcement"]
    )

In [191]:
fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

There was a wrongly entered value, so I looked at the link and corrected it

In [192]:
acquisitions.loc[
    acquisitions["Year of acquisition announcement"] == 2104,
    "Year of acquisition announcement",
] = 2014

Plotting again without the error, now we can see that the overall trend of prices tends to go up, that's why we added the 'Year of acquisitions announcement' column

In [193]:
fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=700,
    height=400,
)
fig.show()

update the datatypes automatically

In [194]:
acquired = acquired.infer_objects()
acquisitions = acquisitions.infer_objects()

In [195]:
fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

Another error found and corrected

In [196]:
acquired.loc[acquired["Year Founded"] == 1840, "Year Founded"] = 2006
acquired.loc[acquired["Year Founded"] == 1933, "Year Founded"] = 1989

In [197]:
fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

In [198]:
acquired.iloc[12]["Tagline"]

'5min Media is a syndication platform for lifestyle, knowledge and instructional videos.'

In [199]:
for l in acquired.iloc[12]["Description"].split("."):
    print(l + "\n")

5min Media is the leading syndication platform for lifestyle, knowledge and instructional videos

 Reinventing the cable network online, 5min reaches engaged and targeted audiences of passionate consumers through its network of 100s of lifestyle and niche websites

 The 5min video library comprises more than 200,000 short-form videos from some of the world's largest media companies, as well as the most innovative independent producers

 Visit  for more information





* 'Tagline' contains a brief and precise description of the company , while the 'Description' is very long and doesn't provide any more important details, 
so we will drop the 'Description'

In [200]:
acquiring = acquiring.drop("Description", axis=1)
acquired = acquired.drop("Description", axis=1)

### There isn't any new useful information that we can get out of those , so we will drop them

* "CrunchBase Profile" and "API" columns are both on the crunchbase website , which uses captcha so we can't scrap it, and their API is paid , and the provided API key is invalid , so we can't use it

* "Homepage" column contains the link to the website of every company , and they aren't all the same so we can't apply a function or a program to extract certain information about them. To use the link , this would require us to go over into each of them one by one , which isn't  feasible


* "Twitter" column also can't be scraped according to their new policy , tried multiple APIs and libraries but none of them worked , even twitter's free tier API is useless
 

* "Acquisition ID" is just used to link between files , and we can do that with the company's name


In [201]:
acquired = acquired.drop(
    ["CrunchBase Profile", "Homepage", "Twitter", "Acquisitions ID", "API"], axis=1
)
acquiring = acquiring.drop(
    ["CrunchBase Profile", "Homepage", "Twitter", "Acquisitions ID", "API"], axis=1
)
founders = founders.drop("CrunchBase Profile", axis=1)

In [202]:
acquired["Age on acquisition"] = (
    acquired["Year of acquisition announcement"] - acquired["Year Founded"]
)

In [203]:
acquired = acquired.drop(["Year Founded", "Year of acquisition announcement"], axis=1)

All these columns are probably related to the target column , so we will keep them for now

Market categories contains multiple values , still not processed

In [204]:
acquired.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company              310 non-null    object 
 1   Tagline              307 non-null    object 
 2   Market Categories    287 non-null    object 
 3   City (HQ)            275 non-null    object 
 4   State / Region (HQ)  273 non-null    object 
 5   Country (HQ)         276 non-null    object 
 6   Acquired by          309 non-null    object 
 7   Price                310 non-null    int64  
 8   Age on acquisition   241 non-null    float64
dtypes: float64(1), int64(1), object(7)
memory usage: 21.9+ KB


Dropping 'year of last update' of the number of employees , because we don't need it directly and can't use it in any way to pridct the current number

In [205]:
acquiring = acquiring.drop("Number of Employees (year of last update)", axis=1)

There are multiple 'NOT YET' in the IPO column , and the earliest the number the better it is , so we won't replace them with zero ,we will replace them with 2025 or anything larger

In [206]:
acquiring["IPO"].value_counts()[:5]

IPO
1986       4
1978       4
1983       2
Not yet    2
1990       2
Name: count, dtype: int64

In [207]:
acquiring.loc[acquiring["IPO"] == "Not yet", "IPO"] = 2025  # 2025 is debatable

In [208]:
acquiring["Number of Employees"] = [
    int(n.replace(",", "")) if type(n) != float else n
    for n in acquiring["Number of Employees"]
]

Idea for acquiring companies: calculate the average price paid for all acquired companies

how to categorize multiple values in the same cell?

In [209]:
acquiring["Market Categories"][:5]

0    Photo Editing, Design, Creative, Software, Ima...
1    Groceries, Consumer Goods, Crowdsourcing, E-Co...
2    News, Advertising Platforms, Content Creators,...
3    Computers, Consumer Electronics, Hardware + So...
4                                               Mobile
Name: Market Categories, dtype: object

In [210]:
acquiring = acquiring.astype(
    {
        "IPO": "float",
    }
)

In [211]:
flattened = [x for item in acquiring["Board Members"].dropna() for x in item.split(",")]

In [212]:
pd.Series(flattened).nunique()

309

In [213]:
len(np.intersect1d(founders["Name"], flattened))

34

Some of the board members are in the founders df , so we won't drop them for now

In [214]:
acquiring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Acquiring Company       36 non-null     object 
 1   Tagline                 36 non-null     object 
 2   Market Categories       36 non-null     object 
 3   Year Founded            36 non-null     int64  
 4   IPO                     35 non-null     float64
 5   Founders                36 non-null     object 
 6   Number of Employees     35 non-null     float64
 7   Total Funding ($)       36 non-null     int64  
 8   Number of Acquisitions  36 non-null     int64  
 9   Board Members           34 non-null     object 
 10  City (HQ)               34 non-null     object 
 11  State / Region (HQ)     33 non-null     object 
 12  Country (HQ)            36 non-null     object 
 13  Acquired Companies      36 non-null     object 
dtypes: float64(2), int64(3), object(9)
memory us

In [215]:
founders["Companies"].value_counts()[:5]

Companies
Microsoft                 21
IBM                       20
Cisco Systems             17
Verizon Communications    15
Nokia                     15
Name: count, dtype: int64

In [216]:
founders["Role"].value_counts()

Role
Board of Directors                    222
Founder                                79
Advisory Board                         72
Board of Directors, Advisory Board      4
Board Observer                          3
Board of Directors, Founder             2
Name: count, dtype: int64

The image of the founder doesn't affect anything at all ... DROPPED

In [217]:
founders = founders.drop("Image", axis=1)

Ready

In [218]:
founders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       382 non-null    object
 1   Role       382 non-null    object
 2   Companies  382 non-null    object
dtypes: object(3)
memory usage: 9.1+ KB


* The specific date which the deal was announced on doesn't matter , what matters is the year so the model can know that inflation affects the price
* The ID doesn't add any new info
* The News and News link don't add any info or details about the acquisition

In [219]:
acquisitions = acquisitions.drop(
    ["Deal announced on", "Acquisitions ID", "News", "News Link"], axis=1
)

In [220]:
acquisitions["Status"].value_counts()

Status
Undisclosed    310
Complete        16
Pending          9
Name: count, dtype: int64

In [221]:
acquisitions["Terms"].value_counts()

Terms
Undisclosed    148
Cash           128
Cash, Stock     36
Stock           24
Name: count, dtype: int64

In [222]:
acquisitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Acquired Company                  336 non-null    object
 1   Acquiring Company                 336 non-null    object
 2   Year of acquisition announcement  336 non-null    int64 
 3   Price                             336 non-null    int64 
 4   Status                            335 non-null    object
 5   Terms                             336 non-null    object
dtypes: int64(2), object(4)
memory usage: 15.9+ KB


### Spliting each multi-valued category to an array of categories

In [223]:
def SplitMultiValuedColumn(df, label):
    df[label] = [
        [
            value.strip() if type(value) == str else value
            for value in str(values).split(",")
        ]
        for values in df[label]
    ]

In [224]:
def getUniqueLabels(df, label):
    uniqueLabels = []
    for labels in df[label].dropna():
        for label in labels:
            if [label] not in uniqueLabels:
                uniqueLabels.append(label)
    return np.ravel(uniqueLabels)

In [225]:
def encodeMultiValuedCategory(df, label: str, categories=[]):
    le = preprocessing.LabelEncoder()
    SplitMultiValuedColumn(df, label)
    if len(categories) == 0:
        categories = getUniqueLabels(df, label)
    le.fit(categories)
    df[label] = [le.transform(values) for values in df[label].dropna()]
    return le.classes_

In [226]:
def encodeCategory(df, label: str, categories=[]):
    nonNullIndex = df[label].notna()

    le = preprocessing.LabelEncoder()
    if len(categories) == 0:
        categories = df.loc[nonNullIndex, label]

    le.fit(categories)
    df.loc[nonNullIndex, label] = le.transform(df.loc[nonNullIndex, label])
    return le.classes_

In [227]:
def FindMultiValuedColumns(df):
    cols = []
    for col in df.columns:
        try:  # To skip numeric columns
            if (
                len(
                    [
                        value
                        for value in df[col].dropna().values
                        if len(value.split(",")) > 1
                    ]
                )
                > 1
            ):
                cols.append(col)
        except:
            pass
    return cols

In [228]:
FindMultiValuedColumns(acquisitions)

['Terms']

In [229]:
FindMultiValuedColumns(acquired)

['Tagline', 'Market Categories']

In [230]:
FindMultiValuedColumns(acquiring)

['Tagline',
 'Market Categories',
 'Founders',
 'Board Members',
 'Acquired Companies']

In [231]:
FindMultiValuedColumns(founders)

['Role', 'Companies']

##### encoding data in Aquisitions

In [232]:
for label in FindMultiValuedColumns(acquisitions):
    print(encodeMultiValuedCategory(acquisitions, label)[:5])

['Cash' 'Stock' 'Undisclosed']


In [233]:
acquisitions["Terms"][:5]

0       [0]
1       [0]
2       [2]
3       [2]
4    [0, 1]
Name: Terms, dtype: object

In [234]:
encodeCategory(acquisitions, "Status")

array(['Complete', 'Pending', 'Undisclosed'], dtype=object)

In [235]:
acquisitions = acquisitions.infer_objects()

In [236]:
acquisitions[:3]

,Acquired Company,Acquiring Company,Year of acquisition announcement,Price,Status,Terms
0,Data Domain,EMC,2009,2100000000,2.0,[0]
1,Quigo,AOL,2007,363000000,2.0,[0]
2,PostPath,Cisco Systems,2008,215000000,2.0,[2]


##### encoding data in Founders

In [237]:
founders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       382 non-null    object
 1   Role       382 non-null    object
 2   Companies  382 non-null    object
dtypes: object(3)
memory usage: 9.1+ KB


In [238]:
foundersNames = encodeCategory(founders, "Name")

In [239]:
foundersNames[:3]

array(['Adam Grosser', 'Akio Morita', 'Al Gore'], dtype=object)

In [240]:
foundersRoles = encodeMultiValuedCategory(founders, "Role")

In [241]:
foundersRoles

array(['Advisory Board', 'Board Observer', 'Board of Directors',
       'Founder'], dtype='<U18')

##### encoding data in Aquiring

In [242]:
acquiringCompanies = encodeMultiValuedCategory(acquiring, "Acquiring Company")
acquiredCompanies = encodeMultiValuedCategory(acquiring, "Acquired Companies")

categories = ["City (HQ)", "State / Region (HQ)", "Country (HQ)"]
multiValuedCategories = ["Market Categories", "Board Members"]
# saving acquiring companies names so i can encode the same column in different dataFrame with the same values
# and there is multiple shared columns in different dataframes that needs this operation

for c in categories:
    print(encodeCategory(acquiring, c)[:3])

for c in multiValuedCategories:
    print(encodeMultiValuedCategory(acquiring, c)[:3])

['Armonk' 'Cupertino' 'Dallas']
['California' 'Massachusetts' 'New York']
['Canada' 'Germany' 'Norway']
['Advertising Platforms' 'All Markets' 'All Students']
['AJ Jaghori' 'Adam Grosser' 'Al Gore']


In [243]:
encodeMultiValuedCategory(acquiring, "Founders", foundersNames)[:2]

array(['Adam Grosser', 'Akio Morita'], dtype=object)

In [244]:
acquiring[:2]

,Acquiring Company,Tagline,Market Categories,Year Founded,IPO,Founders,Number of Employees,Total Funding ($),Number of Acquisitions,Board Members,City (HQ),State / Region (HQ),Country (HQ),Acquired Companies
0,[2],Adobe is an American multinational computer so...,"[38, 19, 16, 49, 31]",1982,1986.0,"[191, 46]",11144.0,0,38,[305],18,0,5,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372..."
1,[3],Amazon is an international e-commerce company ...,"[27, 14, 17, 21]",1994,1997.0,[173],132600.0,8000000,45,"[100, 218, 296, 205, 25, 139, 144, 300]",20,7,5,"[22, 25, 57, 76, 125, 187, 185, 197, 214, 220,..."


##### encoding data in Acquired

In [245]:
encodeCategory(acquired, "Company", acquiredCompanies)

array(['2Web Technologies', '2dehands.be', '2ememain.be', ..., 'threadsy',
       'xkoto', 'zynamics'], shape=(1610,), dtype='<U36')

In [246]:
# i need to make global variables for this categories to get the unique from merging acquiring and acuired data
# something getUniqueValues(acquired["City (HQ)"] + acquiring["City (HQ)"])

for c in ["City (HQ)", "State / Region (HQ)", "Country (HQ)", "Acquired by"]:
    print(encodeCategory(acquired, c)[:3])
encodeMultiValuedCategory(acquired, "Market Categories")[:3]

['Alameda' 'Aliso Viejo' 'Arcueil']
['Andalucia' 'Arkansas' 'Basel-Stadt']
['Australia' 'Canada' 'China']
['AOL' 'AT&T' 'Adobe']


array(['3D', 'Advertising', 'Aerospace'], dtype='<U27')

In [247]:
acquired[:5]

,Company,Tagline,Market Categories,City (HQ),State / Region (HQ),Country (HQ),Acquired by,Price,Age on acquisition
0,354,Day Software develops web applications that al...,[117],8,2,13,2,240000000,NaN
1,404,"Efficient Frontier, an online performance and ...",[1],101,5,15,2,400000000,9.0
2,751,Macromedia is a graphics and web development s...,[117],91,5,15,2,3400000000,13.0
3,845,Neolane is a marketing technology provider dev...,[117],2,19,5,2,600000000,12.0
4,921,Omniture is an online marketing and web analyt...,"[117, 1]",73,44,15,2,1800000000,13.0


### Imputing the null values


In [248]:
def knn_impute_numeric(df: pd.DataFrame, n_neighbors: int = 5) -> pd.DataFrame:

    df_copy = df.copy()

    numeric_cols = df_copy.select_dtypes(include=[float, int]).columns
    numeric_df = df_copy[numeric_cols]

    imputer = KNNImputer(n_neighbors=n_neighbors)
    imputed_array = imputer.fit_transform(numeric_df)

    imputed_df = pd.DataFrame(imputed_array, columns=numeric_cols, index=df_copy.index)
    df_copy[numeric_cols] = imputed_df

    return df_copy

In [249]:
acquisitions.isnull().sum().sum()

np.int64(1)

In [250]:
acquisitions = knn_impute_numeric(acquisitions)

In [251]:
acquisitions.isnull().sum().sum()

np.int64(0)

In [252]:
acquired.isnull().sum().sum()

np.int64(179)

In [253]:
acquired = knn_impute_numeric(acquired.infer_objects())

In [254]:
acquired.isnull().sum()

Company                0
Tagline                3
Market Categories      0
City (HQ)              0
State / Region (HQ)    0
Country (HQ)           0
Acquired by            0
Price                  0
Age on acquisition     0
dtype: int64

In [255]:
acquiring.isnull().sum().sum()

np.int64(7)

In [256]:
acquiring = knn_impute_numeric(acquiring.infer_objects())

In [257]:
acquiring.isnull().sum().sum()

np.int64(0)

In [258]:
founders.isnull().sum().sum()

np.int64(0)

In [259]:
numeric_df = acquired.select_dtypes(include=[float, int])
correlations = numeric_df.drop("Price", axis=1).apply(
    lambda x: x.corr(numeric_df["Price"], method="kendall")
)

In [260]:
correlations

Company                0.014899
City (HQ)             -0.020261
State / Region (HQ)   -0.036828
Country (HQ)           0.164481
Acquired by           -0.046799
Age on acquisition     0.357926
dtype: float64